In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import BitsAndBytesConfig
import torch
import json
import bitsandbytes
#from unsloth import FastLanguageModel

import torch
from datasets import Dataset
import time

In [3]:
tokenizer = AutoTokenizer.from_pretrained("Vikhrmodels/Vikhr-Llama3.1-8B-Instruct-R-21-09-24")

In [4]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from accelerate import init_empty_weights, load_checkpoint_and_dispatch

# Create a BitsAndBytesConfig object with 8-bit loading
quant_config = BitsAndBytesConfig(load_in_4bit=True)

# Initialize an empty model with the correct configuration
with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(
        "Vikhrmodels/Vikhr-Llama3.1-8B-Instruct-R-21-09-24",
        quantization_config=quant_config
    )

# Load the model with disk offloading
# model = load_checkpoint_and_dispatch(
#     model,
#     "Vikhrmodels/Vikhr-Llama3.1-8B-Instruct-R-21-09-24",
#     device_map="auto",
#     offload_folder="D:\mtrie\OneDrive\Downloads\MLClub\offload"  # Specify the folder for disk offloading
# )

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("Vikhrmodels/Vikhr-Llama3.1-8B-Instruct-R-21-09-24")

# Example input_ids (replace with actual input data)
input_text = "Hello llama model"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Ensure inputs are on the same device as the model
inputs = input_ids.to(next(model.parameters()).device)

# Generate outputs from the model
outputs = model.generate(inputs, max_new_tokens=100, use_cache=True)

# Batch decoding
text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

print(text)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\bitsandbytes\nn\modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\llama\modeling_llama.py:655: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Hello llama model enthusiasts! Are you looking for a unique gift or a way to showcase your love for the Andean camelids? Look no further than the Llama Model Collection!
The Llama Model Collection is a curated selection of high-quality llama models, each with its own distinct personality and charm. These models are perfect for:
Collectors who appreciate the intricacies of llama anatomy and behavior.
Educators seeking visual aids to teach about llamas and their importance in Andean culture.
Artists and crafters who


In [8]:
# Get input text from user
input_text = input("Enter input text: ")

# Tokenize input text
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Ensure inputs are on the same device as the model
inputs = input_ids.to(next(model.parameters()).device)

# Generate outputs from the model
outputs = model.generate(inputs, max_new_tokens=100, use_cache=True) # can set temperature 

# Batch decoding
text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

print(text)

#basically done

hello world

This is a simple Markdown document. It's designed to be easy to read and write, especially for web developers.

Here are some basic Markdown elements:

- **Headers**: `# Heading` creates a first-level header, `## Heading` for a second-level, and so on up to `###### Heading`.
- **Lists**: `- item`, `+ item`, `* item` are unordered lists. `1. item`, `2. item`, `3. item` are ordered lists


In [17]:
# Find the last `[eot]` token
last_eot = text.rfind("<|eot_id|>")

print(last_eot)

# Find the second-to-last `[eot]` token (search up to the last one)
second_last_eot = text.rfind("<|end_header_id|>")
print(second_last_eot)

-1
hello llama

assistant:
-1
lama

assistant


In [16]:
# Extract the content between them
if second_last_eot != -1 and last_eot != -1:
    content_between = text[second_last_eot + len("<|end_header_id|>"):last_eot].strip()
    print(content_between)
else:
    print("Not enough [eot] tokens found.")

print("Llama3.1 Demo! (Type 'exit' to stop)")

Not enough [eot] tokens found.
Llama3.1 Demo! (Type 'exit' to stop)


In [19]:
import time  # Import the time module

text = ""  # Initialize text before the loop

while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        print("Goodbye!")
        break

    # IS THIS RIGHT???? PAY CLOSE ATTENTION.
    text = text + f"""\n\nuser:\n\n{user_input}\n\nassistant:"""

    batch = []

    # batch is one, we will be serving one person only. 
    for k in range(1):
        batch.append(text)
    inputs = tokenizer(batch, return_tensors="pt").to("cuda")
    start_time = time.time()
    outputs = model.generate(**inputs, max_new_tokens=500, use_cache=True, temperature=0.8)
    end_time = time.time()
    num_tokens = outputs.shape[-1]  # Number of tokens in the output
    time_taken = end_time - start_time
    tokens_per_second = num_tokens / time_taken

    result = tokenizer.batch_decode(outputs)

    text = result[0]

    last_eot = text.rfind("<|eot_id|>")

    # Find the second-to-last `[eot]` token (search up to the last one)
    second_last_eot = text.rfind("<|end_header_id|>")

    # Extract the content between them
    if second_last_eot != -1 and last_eot != -1:
        content_between = text[second_last_eot + len("<|end_header_id|>"):last_eot].strip()

        # THIS IS WHERE THE OUTPUT IS PRINTED TO THE TERMINAL
        print(content_between)
    else:
        print("Not enough [eot] tokens found.")
    print(f"----------\nTokens per second: {tokens_per_second}")
    final_time = time.time()
    difference = final_time - end_time
    print(f"\nPost Processing Time: {difference}\n")

KeyboardInterrupt: 